In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mplcursors
from scipy.stats import linregress
from scipy import stats

%matplotlib widget

In [ ]:
path = '/home/giovanni/Dottorato/Projects/PhD data/Anhydrite friction and AE/Anhydrite friction and AE/Data/s0186/'
file_name = 's186_final.csv'

df = pd.read_csv(path + file_name, sep='\t', skiprows=[1])

#Find the rec number at which experiment starts and finish
plt.figure(figsize=(6,6))
plt.plot(df.records_na, df.friction_na)
plt.xlabel('record number', fontsize=14)
plt.ylabel('friction', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
plt.title('Pick the rec number at which experiment starts and stops')
plt.show()

df

Frictional healing

In [ ]:
# Definizione dei dati
shs_start = 25000                  # pick the record number at which SHS test starts
shs_stop = 53228                   # pick the record number at which SHS test ends
hold_times = [3, 10, 30, 100, 300, 1000]  # select duration of holds

friction_shs = df.friction_na[shs_start:shs_stop]
time_shs = df.time_s[shs_start:shs_stop]
slip_shs = df.lp_disp_mm[shs_start:shs_stop]
lvdt_ob_shs = df.ob_disp[shs_start:shs_stop]
lt_left_shs = df.ed_left_µm[shs_start:shs_stop]
lt_right_shs = df.ed_right_µm[shs_start:shs_stop]

# Creazione del plot
fig, ax = plt.subplots(figsize=(6, 6))
ax.plot(time_shs, friction_shs, label="Friction")

ax.set_xlabel('time (s)', fontsize=14)
ax.set_ylabel('friction', fontsize=14)

# Abilitare il cursore con tooltip (la freccetta con le coordinate)
cursor = mplcursors.cursor(ax, hover=True)

# Lista per salvare i punti selezionati
selected_points = []
healing = []

# Funzione per salvare i punti con doppio click
def on_double_click(event):
    # Verifica che sia un doppio click e che sia all'interno dell'area del grafico
    if event.dblclick and event.inaxes is not None:
        # Ottieni le coordinate x e y del click nel grafico
        x = event.xdata
        y = event.ydata
        
        # Salva il punto selezionato
        selected_points.append((x, y))
        print(f"Selected point with double-click: time={x}, friction={y}")
        
        # Controlla se ci sono abbastanza punti per calcolare la differenza (almeno 2)
        if len(selected_points) % 2 == 0:
            # Calcola la differenza tra l'ultimo punto e il penultimo
            point_diff = selected_points[-1][1] - selected_points[-2][1]  # differenza di attrito (y)
            healing.append(point_diff)
            print(f"Calculated healing value: {point_diff}")

        # Se abbiamo selezionato tutti i punti, mostra il grafico finale
        if len(healing) == len(hold_times):
            fig_finale()

# Funzione per mostrare il grafico finale della regressione
def fig_finale():
    # Converte la lista di healing in un array numpy
    healing_array = np.array(healing)  # Converte la lista in un array numpy
    print(f"Healing array: {healing_array}")

    # Calcola il logaritmo in base 10 di hold_times
    log_hold_times = np.log10(hold_times)

    # Assicuriamoci che healing sia della stessa lunghezza di hold_times
    if len(healing_array) != len(hold_times):
        print(f"Errore: Il numero di punti di healing ({len(healing_array)}) non corrisponde a quello di hold_times ({len(hold_times)}). Seleziona esattamente {len(hold_times)} coppie di punti.")
    else:
        # Calcolo della regressione lineare
        slope, intercept, r_value, p_value, std_err = stats.linregress(log_hold_times, healing_array)

        # Plot dei punti e della retta di regressione
        fig, ax = plt.subplots(figsize=(6, 6))

        # Plot dei punti
        ax.scatter(log_hold_times, healing_array, color='blue', edgecolors='k', s=100, label="Healing", zorder=5)

        # Calcola i valori della retta di regressione
        regression_line = [slope * x + intercept for x in log_hold_times]

        # Plot della retta di regressione
        ax.plot(log_hold_times, regression_line, color='red', linewidth=3, alpha=0.6, label=f'Healing rate\nβ = {slope:.4f}', zorder=3)

        # Aggiungere etichette e titolo
        ax.set_xlabel('$\mathrm{log_{10}(t_{hold})}$', fontsize=14)
        ax.set_ylabel('Healing, $\mathrm{\Delta\mu}$', fontsize=14)
        ax.set_title('Linear Regression of Healing vs Log10 Hold Time')

        # Aggiungere una legenda
        ax.legend()

        # Impostare l'asse x in scala logaritmica
        ax.set_xscale('linear')  # Questo renderà la visualizzazione della regressione corretta

        # Mostra il plot finale
        plt.show()

        # Stampa il valore di β a schermo
        print(f"Healing rate β : {slope:.4f}")

# Collegare l'evento del doppio click al grafico
fig.canvas.mpl_connect('button_press_event', on_double_click)

# Aggiungi interazione con il cursore per mostrare le coordinate
@cursor.connect("add")
def on_hover(sel):
    sel.annotation.set(text=f'time={sel.target[0]:.2f}\nfriction={sel.target[1]:.2f}')

plt.show()

Creep relaxation

In [ ]:
# Creazione del plot
fig, ax = plt.subplots(figsize=(6, 6))
ax.plot(time_shs, friction_shs, label="Friction")

ax.set_xlabel('time (s)', fontsize=14)
ax.set_ylabel('friction', fontsize=14)

# Abilitare il cursore con tooltip (la freccetta con le coordinate)
cursor = mplcursors.cursor(ax, hover=True)

# Lista per salvare i punti selezionati
selected_points = []
creep_relaxation = []

# Funzione per salvare i punti con doppio click
def on_double_click(event):
    # Verifica che sia un doppio click e che sia all'interno dell'area del grafico
    if event.dblclick and event.inaxes is not None:
        # Ottieni le coordinate x e y del click nel grafico
        x = event.xdata
        y = event.ydata
        
        # Salva il punto selezionato
        selected_points.append((x, y))
        print(f"Selected point with double-click: time={x}, friction={y}")
        
        # Controlla se ci sono abbastanza punti per calcolare la differenza (almeno 2)
        if len(selected_points) % 2 == 0:
            # Calcola la differenza tra l'ultimo punto e il penultimo
            point_diff = selected_points[-1][1] - selected_points[-2][1]  # differenza di attrito (y)
            creep_relaxation.append(point_diff)
            print(f"Calculated creep_relaxation value: {point_diff}")

        # Se abbiamo selezionato tutti i punti, mostra il grafico finale
        if len(creep_relaxation) == len(hold_times):
            fig_finale()

# Funzione per mostrare il grafico finale della regressione
def fig_finale():
    # Converte la lista di creep_relaxation in un array numpy
    creep_relaxation_array = np.array(creep_relaxation)  # Converte la lista in un array numpy
    print(f"creep_relaxation array: {creep_relaxation_array}")

    # Calcola il logaritmo in base 10 di hold_times
    log_hold_times = np.log10(hold_times)

    # Assicuriamoci che creep_relaxation sia della stessa lunghezza di hold_times
    if len(creep_relaxation_array) != len(hold_times):
        print(f"Errore: Il numero di punti di creep_relaxation ({len(creep_relaxation_array)}) non corrisponde a quello di hold_times ({len(hold_times)}). Seleziona esattamente {len(hold_times)} coppie di punti.")
    else:
        # Calcolo della regressione lineare
        slope, intercept, r_value, p_value, std_err = stats.linregress(log_hold_times, creep_relaxation_array)

        # Plot dei punti e della retta di regressione
        fig, ax = plt.subplots(figsize=(6, 6))

        # Plot dei punti
        ax.scatter(log_hold_times, creep_relaxation_array, color='blue', edgecolors='k', s=100, label="creep_relaxation", zorder=5)

        # Calcola i valori della retta di regressione
        regression_line = [slope * x + intercept for x in log_hold_times]

        # Plot della retta di regressione
        ax.plot(log_hold_times, regression_line, color='red', linewidth=3, alpha=0.6, label=f'creep_relaxation rate\nβ = {slope:.4f}', zorder=3)

        # Aggiungere etichette e titolo
        ax.set_xlabel('$\mathrm{log_{10}(t_{hold})}$', fontsize=14)
        ax.set_ylabel('creep_relaxation, $\mathrm{\Delta\mu}$', fontsize=14)
        ax.set_title('Linear Regression of creep_relaxation vs Log10 Hold Time')

        # Aggiungere una legenda
        ax.legend()

        # Impostare l'asse x in scala logaritmica
        ax.set_xscale('linear')  # Questo renderà la visualizzazione della regressione corretta

        # Mostra il plot finale
        plt.show()

        # Stampa il valore di β a schermo
        print(f"creep_relaxation rate β : {slope:.4f}")

# Collegare l'evento del doppio click al grafico
fig.canvas.mpl_connect('button_press_event', on_double_click)

# Aggiungi interazione con il cursore per mostrare le coordinate
@cursor.connect("add")
def on_hover(sel):
    sel.annotation.set(text=f'time={sel.target[0]:.2f}\nfriction={sel.target[1]:.2f}')

plt.show()  # Mostra il primo grafico

Hold compaction

In [ ]:
# Creazione del plot
fig, ax = plt.subplots(figsize=(6, 6))
ax.plot(time_shs, lt_left_shs, label="Layer thickness, left layer")
#ax.plot(time_shs, lt_right_shs, label="Layer thickness, right layer")


ax.set_xlabel('time (s)', fontsize=14)
ax.set_ylabel('layer thickness (micrometers)', fontsize=14)

# Abilitare il cursore con tooltip (la freccetta con le coordinate)
cursor = mplcursors.cursor(ax, hover=True)

# Lista per salvare i punti selezionati
selected_points = []
compaction = []

# Funzione per salvare i punti con doppio click
def on_double_click(event):
    # Verifica che sia un doppio click e che sia all'interno dell'area del grafico
    if event.dblclick and event.inaxes is not None:
        # Ottieni le coordinate x e y del click nel grafico
        x = event.xdata
        y = event.ydata
        
        # Salva il punto selezionato
        selected_points.append((x, y))
        print(f"Selected point with double-click: time={x}, layer thickness={y}")
        
        # Controlla se ci sono abbastanza punti per calcolare la differenza (almeno 2)
        if len(selected_points) % 2 == 0:
            # Calcola la differenza tra l'ultimo punto e il penultimo
            point_diff = selected_points[-1][1] - selected_points[-2][1]  # differenza di attrito (y)
            compaction.append(point_diff)
            print(f"Calculated compaction value: {point_diff}")

        # Se abbiamo selezionato tutti i punti, mostra il grafico finale
        if len(compaction) == len(hold_times):
            fig_finale()

# Funzione per mostrare il grafico finale della regressione
def fig_finale():
    # Converte la lista di compaction in un array numpy
    compaction_array = np.array(compaction)  # Converte la lista in un array numpy
    print(f"compaction array: {compaction_array}")

    # Calcola il logaritmo in base 10 di hold_times
    log_hold_times = np.log10(hold_times)

    # Assicuriamoci che compaction sia della stessa lunghezza di hold_times
    if len(compaction_array) != len(hold_times):
        print(f"Errore: Il numero di punti di compaction ({len(compaction_array)}) non corrisponde a quello di hold_times ({len(hold_times)}). Seleziona esattamente {len(hold_times)} coppie di punti.")
    else:
        # Calcolo della regressione lineare
        slope, intercept, r_value, p_value, std_err = stats.linregress(log_hold_times, compaction_array)

        # Plot dei punti e della retta di regressione
        fig, ax = plt.subplots(figsize=(6, 6))

        # Plot dei punti
        ax.scatter(log_hold_times, compaction_array, color='blue', edgecolors='k', s=100, label="compaction", zorder=5)

        # Calcola i valori della retta di regressione
        regression_line = [slope * x + intercept for x in log_hold_times]

        # Plot della retta di regressione
        ax.plot(log_hold_times, regression_line, color='red', linewidth=3, alpha=0.6, label=f'compaction rate\nβ = {slope:.4f}', zorder=3)

        # Aggiungere etichette e titolo
        ax.set_xlabel('$\mathrm{log_{10}(t_{hold})}$', fontsize=14)
        ax.set_ylabel('compaction, $\mathrm{\Delta\mu}$', fontsize=14)
        ax.set_title('Linear Regression of compaction vs Log10 Hold Time')

        # Aggiungere una legenda
        ax.legend()

        # Impostare l'asse x in scala logaritmica
        ax.set_xscale('linear')  # Questo renderà la visualizzazione della regressione corretta

        # Mostra il plot finale
        plt.show()

        # Stampa il valore di β a schermo
        print(f"compaction rate C : {slope:.4f}")

# Collegare l'evento del doppio click al grafico
fig.canvas.mpl_connect('button_press_event', on_double_click)

# Aggiungi interazione con il cursore per mostrare le coordinate
@cursor.connect("add")
def on_hover(sel):
    sel.annotation.set(text=f'time={sel.target[0]:.2f}\nlayer thickness={sel.target[1]:.2f}')

plt.show()  # Mostra il primo grafico

Reshear dilatancy

In [ ]:
# Creazione del plot
fig, ax = plt.subplots(figsize=(6, 6))
ax.plot(time_shs, lt_left_shs, label="Friction")

ax.set_xlabel('time (s)', fontsize=14)
ax.set_ylabel('layer thickness (mm)', fontsize=14)

# Abilitare il cursore con tooltip (la freccetta con le coordinate)
cursor = mplcursors.cursor(ax, hover=True)

# Lista per salvare i punti selezionati
selected_points = []
dilatancy = []

# Funzione per salvare i punti con doppio click
def on_double_click(event):
    # Verifica che sia un doppio click e che sia all'interno dell'area del grafico
    if event.dblclick and event.inaxes is not None:
        # Ottieni le coordinate x e y del click nel grafico
        x = event.xdata
        y = event.ydata
        
        # Salva il punto selezionato
        selected_points.append((x, y))
        print(f"Selected point with double-click: time={x}, friction={y}")
        
        # Controlla se ci sono abbastanza punti per calcolare la differenza (almeno 2)
        if len(selected_points) % 2 == 0:
            # Calcola la differenza tra l'ultimo punto e il penultimo
            point_diff = selected_points[-1][1] - selected_points[-2][1]  # differenza di attrito (y)
            dilatancy.append(point_diff)
            print(f"Calculated dilatancy value: {point_diff}")

        # Se abbiamo selezionato tutti i punti, mostra il grafico finale
        if len(dilatancy) == len(hold_times):
            fig_finale()

# Funzione per mostrare il grafico finale della regressione
def fig_finale():
    # Converte la lista di dilatancy in un array numpy
    dilatancy_array = np.array(dilatancy)  # Converte la lista in un array numpy
    print(f"dilatancy array: {dilatancy_array}")

    # Calcola il logaritmo in base 10 di hold_times
    log_hold_times = np.log10(hold_times)

    # Assicuriamoci che dilatancy sia della stessa lunghezza di hold_times
    if len(dilatancy_array) != len(hold_times):
        print(f"Errore: Il numero di punti di dilatancy ({len(dilatancy_array)}) non corrisponde a quello di hold_times ({len(hold_times)}). Seleziona esattamente {len(hold_times)} coppie di punti.")
    else:
        # Calcolo della regressione lineare
        slope, intercept, r_value, p_value, std_err = stats.linregress(log_hold_times, dilatancy_array)

        # Plot dei punti e della retta di regressione
        fig, ax = plt.subplots(figsize=(6, 6))

        # Plot dei punti
        ax.scatter(log_hold_times, dilatancy_array, color='blue', edgecolors='k', s=100, label="dilatancy", zorder=5)

        # Calcola i valori della retta di regressione
        regression_line = [slope * x + intercept for x in log_hold_times]

        # Plot della retta di regressione
        ax.plot(log_hold_times, regression_line, color='red', linewidth=3, alpha=0.6, label=f'dilatancy rate\nβ = {slope:.4f}', zorder=3)

        # Aggiungere etichette e titolo
        ax.set_xlabel('$\mathrm{log_{10}(t_{hold})}$', fontsize=14)
        ax.set_ylabel('dilatancy, $\mathrm{\Delta\mu}$', fontsize=14)
        ax.set_title('Linear Regression of dilatancy vs Log10 Hold Time')

        # Aggiungere una legenda
        ax.legend()

        # Impostare l'asse x in scala logaritmica
        ax.set_xscale('linear')  # Questo renderà la visualizzazione della regressione corretta

        # Mostra il plot finale
        plt.show()

        # Stampa il valore di β a schermo
        print(f"dilatancy rate β : {slope:.4f}")

# Collegare l'evento del doppio click al grafico
fig.canvas.mpl_connect('button_press_event', on_double_click)

# Aggiungi interazione con il cursore per mostrare le coordinate
@cursor.connect("add")
def on_hover(sel):
    sel.annotation.set(text=f'time={sel.target[0]:.2f}\nlayer thickness={sel.target[1]:.2f}')

plt.show()  # Mostra il primo grafico

Data exportation

In [ ]:
data = {
'hold_time': hold_times,
'healing': healing,
'creep_relaxation': creep_relaxation,  
'compaction': compaction,
'dilatancy': dilatancy                   
}

df = pd.DataFrame(data)
df.to_csv('shs_data_s0186.csv', index=False)

dt = pd.read_csv('shs_data_s0186.csv')
dt